In [ ]:
!pip install -q -U langchain transformers sentence-transformers faiss-cpu torch accelerate bitsandbytes huggingface_hub
!pip install -U langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.4/563.4 kB 19.3 MB/s eta 0:00:00


In [ ]:
! pip install flask-ngrok flask

In [ ]:
!pip install -q ctransformers langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 102.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok

In [ ]:
import json
import textwrap
from langchain_community.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from huggingface_hub import login

In [ ]:
print("Please log in to your Hugging Face account.")
login()

Please log in to your Hugging Face account.


In [ ]:
! pip install jq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.1/757.1 kB 14.4 MB/s eta 0:00:00


In [ ]:
print("\n--- Loading and Processing Knowledge Base ---")
loader = JSONLoader(
    file_path='./dieases.json',
    jq_schema='.',
    text_content=False,
)
documents = loader.load()


--- Loading and Processing Knowledge Base ---


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
docs = text_splitter.split_documents(documents)

In [ ]:
print("Loading embedding model...")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

Loading embedding model...


/tmp/ipython-input-2937718761.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
print("Creating FAISS vector store...")
db = FAISS.from_documents(docs, embeddings)

Creating FAISS vector store...


In [ ]:
retriever = db.as_retriever(search_kwargs={"k": 3})

In [ ]:
print("\n--- Loading Llama 3 8B Instruct Model ---")
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto", # Automatically use the GPU if available
)




--- Loading Llama 3 8B Instruct Model ---


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Device set to use cuda:0


Model loaded successfully!


/tmp/ipython-input-2940379137.py:20: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [ ]:
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    model_kwargs={"temperature": 0.1, "max_new_tokens": 512}, # Pass model-specific arguments here
    return_full_text=False # Crucial: This tells the pipeline to only return the generated answer
)
llm = HuggingFacePipeline(pipeline=pipe)
print("Model loaded successfully!")

Device set to use cuda:0


Model loaded successfully!


In [ ]:

print("\n--- Setting up the RAG Chain ---")

prompt_template = """
### INSTRUCTIONS ###
You are a helpful and polite medical health assistant. Your job is to answer health-related questions accurately based ONLY on the information provided in the context below.
- If the context does not contain enough information to answer the question, you MUST state "I'm sorry, I don't have enough information to answer that question based on my current knowledge base."
- Do not make up information or use any knowledge outside of the provided context.
- Your answers should be clear, concise, and easy for a non-medical person to understand.
- try to respond relatable with the question following context you get

### CONTEXT ###
{context}

### QUESTION ###
{question}

### HELPFUL ANSWER ###
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])


--- Setting up the RAG Chain ---


In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=False, # Set to True if you want to see the retrieved text chunks
    chain_type_kwargs={"prompt": prompt}
)

In [ ]:
! pip install pyngrok

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok, conf
import os

# --- PASTE YOUR NGROK AUTHTOKEN HERE ---
# It's better to add this as a secret in Colab for security
NGROK_AUTHTOKEN = "32slYmXPQSoqvFAu8zakxn13ftS_3Rc3Xf9tiWCi2sMbKxPJA"

In [ ]:
app = Flask(__name__)
ngrok.set_auth_token(NGROK_AUTHTOKEN)
ngrok.kill()
public_url = ngrok.connect(5000).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")

@app.route("/ask", methods=['POST'])
def ask_chatbot():
    data = request.get_json()
    question = data.get("question", "")
    if not question:
        return jsonify({"error": "No question provided"}), 400

    print(f"Received question: {question}")
    result = qa_chain.invoke(question)
    print(f"Generated answer: {result['result']}")

    return jsonify({"answer": result['result']})

if __name__ == "__main__":
    app.run(port=5000)


 * ngrok tunnel "https://14834e30e2bc.ngrok-free.app" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Received question: how to live a healthy lifestyle?


INFO:werkzeug:127.0.0.1 - - [18/Sep/2025 18:39:11] "POST /ask HTTP/1.1" 200 -


Generated answer: I'd be happy to help!

Living a healthy lifestyle can be achieved by incorporating simple habits into your daily routine. Based on the context provided, I can suggest the following tips:

1. Maintain facial hygiene: For oily skin, practice good hygiene by washing your face twice a day with a gentle cleanser and moisturizing regularly.
2. Eat a balanced diet: Avoid high glycemic foods and dairy products, which can trigger hormonal changes and acne. Focus on whole, nutritious foods like fruits, vegetables, whole grains, and lean proteins.
3. Exercise regularly: Regular physical activity can help reduce stress, improve mood, and boost overall health. Aim for at least 30 minutes of moderate-intensity exercise per day.
4. Practice good posture: Maintaining correct posture can help reduce the risk of neck and back injuries, which can lead to chronic pain and discomfort.
5. Get enough sleep: Aim for 7-8 hours of sleep per night to help your body recover and recharge. Establi

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Received question: tell me about hypertension


INFO:werkzeug:127.0.0.1 - - [18/Sep/2025 18:43:52] "POST /ask HTTP/1.1" 200 -


Generated answer: Based on the context, I can provide information about hypertension.

Hypertension, also known as high blood pressure, is a long-term medical condition where the blood pressure in the arteries is persistently elevated. It does not typically cause symptoms, but it can lead to various complications if left untreated.

The causes and risk factors of hypertension include genetic predisposition, obesity, high salt intake, physical inactivity, and other factors.

The symptoms of hypertension may include headache, dizziness, lightheadedness, blurred vision, double vision, nosebleeds, shortness of breath, chest pain, palpitations, fatigue, weakness, confusion, nausea, vomiting, anxiety, restlessness, buzzing in ears (tinnitus), irregular heartbeat, sweating, trouble sleeping, and irregular heartbeat.

To diagnose hypertension, healthcare professionals use physical examination, blood pressure measurement, and other tests.

To prevent or manage hypertension, maintaining a health